In [907]:
#Imports
import spotipy, json, webbrowser, urllib.request, random
import pandas as pd
import spotipy.util as util
from PIL import Image
from qrcode import QRCode
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.utils import ImageReader
from reportlab.platypus import Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate
from reportlab.lib.units import mm

In [908]:
#User Input Cell 1
kseniia_lucky_number = 5
omotara_lucky_number = 3

# #Manual input, comment the below lines out when ready
# user_input = [["Dynamic", "Spicy"],  ["Beef"], ["Thai", "Indian", "Korean"], [28, 11, 23], "K-pop"]
# date_vibe = user_input[0]
# dietary_req = user_input[1]
# pref_cuisines = user_input[2]
# date_day = user_input[3][0]
# date_month = user_input[3][1]
# date_year = user_input[3][2]
# date_genre = user_input[4]

#Dynamic input, uncomment the below lines later
date_vibe = input("Tell me about your date... \n What vibe do you want? \n Describe it in one word (e.g. Adventure")
dietary_req = input("And what are your dietary requirements? \n (e.g. Vegan, Beef)\n").split(", ")
pref_cuisines = input("What are your preferred cuisines? \n (e.g. Italian, Korean)\n").split(", ")
true_date = input("When is the date? \n Format it like this dd, mm, yy (e.g. 28, 11, 23)\n").split(", ")
date_genre = input("And finally, what genre would you like to listen to? Give one option")
date_day = true_date[0]
date_month = true_date[1]
date_year = true_date[2]


In [909]:
#Cocktail Cell 1
letters = []
for word in date_vibe:
    for char in word:
        letters.append(char.lower())

print(letters)

['c', 'h', 'i', 'l', 'l']


In [910]:
#Cocktail Cell 2
cocktail_suggestions = []
for letter in letters:
    number = kseniia_lucky_number
    query_str = f"http://www.thecocktaildb.com/api/json/v1/1/search.php?f={letter}"
    request = urllib.request.Request(query_str)
    response = urllib.request.urlopen(request)
    all_cocktails = json.loads(response.read())
    if all_cocktails['drinks'] != None:
        if len(all_cocktails['drinks']) < (number-1):
            number = len(all_cocktails['drinks'])
        cocktail = all_cocktails['drinks']
        if cocktail[0]['strDrink'] not in cocktail_suggestions:
            cocktail_name = cocktail[0]['strDrink']
            cocktail_instructions = cocktail[0]['strInstructions']
            cocktail_ingred = cocktail[0]['strIngredient1'].lower()
            cocktail_img = cocktail[0]['strDrinkThumb']
            num = 2
            while cocktail[0][f'strIngredient{num}'] != None:
                cocktail_ingred = cocktail_ingred + ", " + cocktail[0][f'strIngredient{num}'].lower()
                num += 1
            cocktail_suggestions.append([cocktail_name, cocktail_instructions, cocktail_ingred, cocktail_img])
print(cocktail_suggestions)

[['Casino', 'Pour all ingredients into shaker with ice cubes. Shake well. Strain into chilled cocktail glass. Garnish with a lemon twist and a maraschino cherry. Serve without a straw.', 'gin, maraschino liqueur, lemon juice, orange bitters, cherry', 'https://www.thecocktaildb.com/images/media/drink/1mvjxg1504348579.jpg'], ['H.D.', 'Mix the whisky and Baileys Cream in a beer-glass (at least 50 cl). Fill the rest of the glass with coffee.', 'whisky, baileys irish cream, coffee', 'https://www.thecocktaildb.com/images/media/drink/upusyu1472667977.jpg'], ['Ipamena', 'Cut half a lime into pieces, place in a shaker, add the sugar and crush. Measure the passion fruit juice, add it to the shaker and fill up with ice cubes. Close the shaker and shake vigorously. Pour the liquid into a glass, top up with ginger ale, stir with a teaspoon and then garnish the rim of the glass with a slice of lime', 'lime, brown sugar, passion fruit juice, ginger ale, ice', 'https://www.thecocktaildb.com/images/med

In [911]:
#Cocktail Cell 3
number = omotara_lucky_number
random.shuffle(cocktail_suggestions)
if len(cocktail_suggestions) < number:
    number = len(cocktail_suggestions)
your_cocktail = cocktail_suggestions[number][0]
your_recipe = cocktail_suggestions[number][1]
your_ingredients = cocktail_suggestions[number][2]
your_cocktail_img = cocktail_suggestions[number][3]

a_or_an = "a"
vowels = ["a", "e", "i", "o", "u"]
if your_cocktail[0].lower() in vowels:
    a_or_an = "an"
    
print(f"Your cocktail for the evening is {a_or_an}\033[1m {your_cocktail}\n\033[0m Here's the recipe \n Ingredients: {your_ingredients}\n Steps: {your_recipe}")

Your cocktail for the evening is a H.D.
 Here's the recipe 
 Ingredients: whisky, baileys irish cream, coffee
 Steps: Mix the whisky and Baileys Cream in a beer-glass (at least 50 cl). Fill the rest of the glass with coffee.


In [912]:
#Meal Cell 1

cuisine_suggestions = []
valid_recipes = []
alphabet = "abcdefghijklmnopqrstuvwxyz"

for user_cuisine in pref_cuisines:
    query_str = f"https://www.themealdb.com/api/json/v1/1/filter.php?a={user_cuisine}"
    request = urllib.request.Request(query_str)
    response = urllib.request.urlopen(request)
    all_responses_for_cuisine = json.loads(response.read())
    all_meals_for_cuisine = all_responses_for_cuisine['meals']
    if all_meals_for_cuisine != None:
        for dish in all_meals_for_cuisine:
            dish_id = dish['idMeal']
            query_str = f"https://www.themealdb.com/api/json/v1/1/lookup.php?i={dish_id}"
            request = urllib.request.Request(query_str)
            response = urllib.request.urlopen(request)
            all_dish_responses = json.loads(response.read())
            dish_info = all_dish_responses['meals']
            
            for req in dietary_req:
                if dish_info[0]['strCategory'] == req:
                    meal = dish_info[0]['strMeal']
                    category = dish_info[0]['strCategory']
                    area = dish_info[0]['strArea']
                    instruct = dish_info[0]['strInstructions']
                    image = dish_info[0]['strMealThumb']
                    meal_ingred = dish_info[0]['strIngredient1'].lower()
                    for num in range(2, 20):
                        if dish_info[0][f'strIngredient{num}'] != None and dish_info[0][f'strIngredient{num}'] != "":
                            meal_ingred = meal_ingred + ", " + dish_info[0][f'strIngredient{num}'].lower()
                            print(dish_info[0][f'strIngredient{num}'], type(dish_info[0][f'strIngredient{num}']))
                            print(meal_ingred)
                    ls = [meal, category, area, instruct, image, meal_ingred]
                    valid_recipes.append(ls)
    else:
        print(f"There were no meals for the {user_cuisine} cuisine")
                    
if len(valid_recipes) == 0:
    print("No recipes found for any of your preferred cuisines :( Please try another!")
else: 
    your_meal = random.choice(valid_recipes)
    print(your_meal)
    print(len(your_meal))

No recipes found for any of your preferred cuisines :( Please try another!


In [913]:
#Meal Cell 2

your_meal_name = your_meal[0]
your_meal_recipe = your_meal[3]
your_meal_img = your_meal[4]
your_meal_ingredients = your_meal[5]

a_or_an_meal = "a"
vowels = ["a", "e", "i", "o", "u"]
if your_meal[0].lower() in vowels:
    a_or_an_meal = "an"

print(your_meal)

['Katsu Chicken curry', 'Chicken', 'Japanese', 'Prep:15min  ›  Cook:30min  ›  Ready in:45min \r\n\r\nFor the curry sauce: Heat oil in medium non-stick saucepan, add onion and garlic and cook until softened. Stir in carrots and cook over low heat for 10 to 12 minutes.\r\nAdd flour and curry powder; cook for 1 minute. Gradually stir in stock until combined; add honey, soy sauce and bay leaf. Slowly bring to the boil.\r\nTurn down heat and simmer for 20 minutes or until sauce thickens but is still of pouring consistency. Stir in garam masala. Pour the curry sauce through a sieve; return to saucepan and keep on low heat until ready to serve.\r\nFor the chicken: Season both sides of chicken breasts with salt and pepper. Place flour, egg and breadcrumbs in separate bowls and arrange in a row. Coat the chicken breasts in flour, then dip them into the egg, then coat in breadcrumbs, making sure you cover both sides.\r\nHeat oil in large frying pan over medium-high heat. Place chicken into hot o

In [914]:
#Playlist Cell 1

credentials = "spotipy_keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [915]:
#Playlist Cell 2

client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirect_uri = api_tokens["redirect"]
username = api_tokens["username"]

scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public'
auth = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

sp = spotipy.Spotify(auth=auth)

In [916]:
#Playlist Cell 3

track_results = sp.search(q=f'genre:{date_genre}', type='track', limit=10)

In [917]:
#Playlist Cell 4

tracks = track_results['tracks']['items']
track_selection_list = []

for song in tracks:
    track_selection_list.append(song['uri'])

my_playlist = sp.user_playlist_create(user=username, name='Date Night Playlist ♡', public=True, description=f"Enjoy your {date_vibe.lower()} date!")
results = sp.user_playlist_add_tracks(username, my_playlist['id'], track_selection_list)
playlist_link = my_playlist['external_urls']['spotify']


In [918]:
#Playlist Cell 5

webbrowser.open(my_playlist['external_urls']['spotify'])

True

In [919]:
#PDF

top_line_pos = 755
line_spacing = 15
normal_fontsize = 8
h3_fontsize = 9.5
h2_fontsize = 13
h1_fontsize = 21
date_details = date_genre + ' ' + str(date_day) + ' ' + str(date_month) + ' ' + str(date_year)

date_canvas = canvas.Canvas(f"date_results/{date_details.replace(" ", "_")}_date_night.pdf", pagesize=A4)

x_pos_centre_align = date_canvas._pagesize[0] / 2

date_canvas.setFont("Helvetica-Bold", h1_fontsize)
date_canvas.setFillColorRGB(0, 0, 0)
date_canvas.drawCentredString(x_pos_centre_align, top_line_pos, "MENU")

date_canvas.setFont("Helvetica", normal_fontsize)
print(date_canvas.getAvailableFonts())

def line_y_position(line_idx):
    return top_line_pos-(line_idx*line_spacing)

def line_break_str(your_string):
    line = ""
    full_str = []
    line_width = 120
    x = line_width
    if line_width < len(your_string):
        first_space = your_string[(line_width-10):(line_width+10)].index(" ") + line_width - 10
        line = your_string[0:first_space]
        full_str.append(line)
        x = first_space
    else:
        line = your_string

    while x < len(your_string):
        y = x+line_width
        if y < len(your_string):
            index_nearest_space = your_string[(y-10):(y+10)].index(" ") + y - 10
        else:
            index_nearest_space = len(your_string)
        line = your_string[x:index_nearest_space]
        x = index_nearest_space
        full_str.append(line)
    
    return full_str

#Cocktail Section
cocktail_img_item = ImageReader(your_cocktail_img)
position = 3
date_canvas.setFont("Helvetica", h2_fontsize)
date_canvas.drawString(100, line_y_position(position), f"Cocktail: {your_cocktail}")
position += 0.5
date_canvas.setFont("Helvetica", normal_fontsize)
position += 1
date_canvas.drawImage(cocktail_img_item, (x_pos_centre_align-50), line_y_position(position + 6.5), mask='auto', width=100, height=100)
position += 8
date_canvas.setFont("Helvetica-Bold", h3_fontsize)
date_canvas.drawString(100, line_y_position(position), f"Ingredients:")
position += 1
date_canvas.setFont("Helvetica", normal_fontsize)
date_canvas.drawString(100, line_y_position(position), f"{your_ingredients}")
position += 1

position += 1
date_canvas.setFont("Helvetica-Bold", h3_fontsize)
date_canvas.drawString(100, line_y_position(position), f"Steps:")
position += 1
date_canvas.setFont("Helvetica", normal_fontsize)
date_canvas.drawString(100, line_y_position(position), f"{your_recipe}")
position += 1

#Meal Section
meal_img_item = ImageReader(your_meal_img)
position += 2
date_canvas.setFont("Helvetica", h2_fontsize)
date_canvas.drawString(100, line_y_position(position), f"Meal: {your_meal_name}")
date_canvas.setFont("Helvetica", normal_fontsize)
position += 0.5
position += 1
date_canvas.drawImage(meal_img_item, (x_pos_centre_align-50), line_y_position(position + 6.5), mask='auto', width=100, height=100)
position += 8
date_canvas.setFont("Helvetica-Bold", h3_fontsize)
date_canvas.drawString(100, line_y_position(position), f"Ingredients:")
position += 1
date_canvas.setFont("Helvetica", normal_fontsize)

meal_ingredients_broken =  line_break_str(your_meal_ingredients)
for line in meal_ingredients_broken:
    date_canvas.drawString(100, line_y_position(position), f"{line}")
    position += 1

date_canvas.setFont("Helvetica-Bold", h3_fontsize)
position += 1
date_canvas.drawString(100, line_y_position(position), f"Steps:")
position += 1
date_canvas.setFont("Helvetica", normal_fontsize)

meal_recipe_broken =  line_break_str(your_meal_recipe)
for line in meal_recipe_broken:
    date_canvas.drawString(100, line_y_position(position), f"{line.replace("\n", "")}")
    position += 1

#QR Section
qr = QRCode(version=1, box_size=10, border=5)
qr.add_data(playlist_link)
qr.make(fit=True)
img = qr.make_image(fill_color="black", back_color="white")
qr_filename = f"images/qr_code_{date_details.replace(" ", "_")}.png"
img.save(qr_filename)

qr_img_item = ImageReader(f"{qr_filename}")

date_canvas.drawImage(qr_img_item, (x_pos_centre_align-50), line_y_position(position + 6.5), mask='auto', width=100, height=100)
position += 7

#Run
date_canvas.showPage()
date_canvas.save()

['Courier', 'Courier-Bold', 'Courier-BoldOblique', 'Courier-Oblique', 'Helvetica', 'Helvetica-Bold', 'Helvetica-BoldOblique', 'Helvetica-Oblique', 'Symbol', 'Times-Bold', 'Times-BoldItalic', 'Times-Italic', 'Times-Roman', 'ZapfDingbats']
